In [585]:
import random as rand
import copy as copy
from operator import attrgetter
from time import time
from sortedcontainers import SortedList
import bisect
from itertools import accumulate

In [549]:
def getRandomLevel(p = 0.5):
    lvl = 0
    while rand.random() < p and lvl < float('inf'):
        lvl = lvl + 1
    return lvl

class MinMaxList:
    def __init__(self):
        self.nodes = []
        self.min = float("inf")
        self.max = float("-inf")
        self.height = 0

class TeleportList:
    def __init__(self):
        self.p = 0.5
        self.height = getRandomLevel(self.p)
        self.hedges = [MinMaxList() for i in range(self.height + 1)]

    def lookup_dumb(self, nr):
        for he in self.hedges:
            if he.min <= nr <= he.max:
                L = len(he.nodes)
                i = bisect.bisect_left(he.nodes, nr)
                if i != L and he.nodes[i] == nr:
                    return True
        return False

    def lookup_subroutine(self, he, nr):
        L = len(he.nodes)
        i = bisect.bisect_left(he.nodes, nr)
        if i != L and he.nodes[i] == nr:
            return True
        else:
            return False

    def lookup(self, nr):
        for i in self.hedges:
            if i.min > nr:
                return False
            elif nr == i.min or nr == i.max:
                return True
            else:
                if i.max >= nr:
                    L = len(i.nodes)
                    j = bisect.bisect_left(i.nodes, nr)
                    if j != L and i.nodes[j] == nr:
                        return True

    def findsert(self, nr, he):
        L = len(he.nodes)
        i = bisect.bisect_left(he.nodes, nr)
        if i != L and he.nodes[i] == nr:
            return False, False
        else:
            return i, he.nodes

    def insert(self, nr):

        height = getRandomLevel()

        if self.height < height:

            for i in range(height - self.height):
                self.hedges.append(MinMaxList())

            self.height = height

        test = self.hedges[height - 1]

        i, he = self.findsert(nr, test)

        if i is not False:
            he.insert(i, nr)
            if test.min > nr:
                test.min = nr
            if test.max < nr:
                test.max = nr
            self.hedges.sort(key=attrgetter('min'))
        else:
            return False

    def show_hedges(self):
        for i in self.hedges:
            print(i.nodes)

    def show_minmax(self):
        for i in self.hedges:
            print(f'min = {i.min} \nmax = {i.max}')


In [422]:
class NodeSKP(object):
    def __init__(self, key, level):
        self.key = key
        self.forward = [None]*(level+1)

class SkipListNaive(object):
    def __init__(self, max_lvl, P):
        self.MAXLVL = max_lvl
        self.P = P
        self.header = self.createNode(self.MAXLVL, -1)
        self.level = 0

    def createNode(self, lvl, key):
        n = NodeSKP(key, lvl)
        return n

    def randomLevel(self):
        lvl = 0
        while rand.random()<self.P and lvl<self.MAXLVL:lvl += 1
        return lvl

    def find(self, key):
        current = self.header
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
        current = current.forward[0]
        if current and current.key == key:
            return key

    def insert(self, key):
        update = [None]*(self.MAXLVL+1)
        current = self.header
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current

        current = current.forward[0]

        if current is None or current.key != key:
            rlevel = self.randomLevel()

            if rlevel > self.level:
                for i in range(self.level+1, rlevel+1):
                    update[i] = self.header
                self.level = rlevel

            n = self.createNode(rlevel, key)

            for i in range(rlevel+1):
                n.forward[i] = update[i].forward[i]
                update[i].forward[i] = n

In [587]:
hgraph = TeleportList()
hgraph.p = 0.5
skeepNaive = SkipListNaive(10, 0.5)
slist = SortedList()

nr = 100000

ten_thousand_integers = [rand.randint(1, 2000000) for i in range(nr)]

def ten_k_hg(HG):
    for i in range(nr):
        HG.insert(ten_thousand_integers[i])

def ten_k_sl(sl):
    for i in range(nr):
        sl.add(ten_thousand_integers[i])

def ten_k_skeep(skeep):
    for i in range(nr):
        skeep.insert(ten_thousand_integers[i])

%timeit -r 1 -n 1 ten_k_hg(hgraph)
%timeit -r 1 -n 1 ten_k_skeep(skeepNaive)
%timeit -r 1 -n 1 ten_k_sl(slist)

621 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.15 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
166 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [590]:
def ten_k_hg(HG):
    for i in range(nr):
        HG.lookup(ten_thousand_integers[i])

def ten_k_hg_lookup_dumb(HG):
    for i in range(nr):
        HG.lookup_dumb(ten_thousand_integers[i])

def ten_k_skeep(skeep):
    for i in range(nr):
        skeep.find(ten_thousand_integers[i])

def ten_k_sl(sl):
    for i in range(nr):
        ten_thousand_integers[i] in sl

%timeit -r 1 -n 1 ten_k_hg(hgraph)
%timeit -r 1 -n 1 ten_k_hg_lookup_dumb(hgraph)
%timeit -r 1 -n 1 ten_k_skeep(skeepNaive)
%timeit -r 1 -n 1 ten_k_sl(slist)

882 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
667 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.68 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
116 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [548]:
hgraph.show_hedges()

[23896, 170718, 221645, 229694, 308302, 636022, 841494, 1028133, 1119877, 1237642, 1403851, 1421175, 1622180, 1641284, 1687032, 1790836, 1793317]
[43010, 343684, 858514, 1351067, 1593411, 1680660, 1711825, 1719791, 1906725, 1927100]
[65911, 147436, 152351, 193420, 202429, 264321, 331148, 537136, 554897, 558010, 696805, 943720, 1010683, 1114930, 1655903, 1985831]
[81439, 211609, 459873, 1027366, 1238701, 1414001]
[106295, 342204, 488908, 847990, 964992, 978342, 1013370, 1018935, 1090526, 1132515, 1591227, 1665980, 1848419, 1868673, 1963394, 1987491]
[181775, 590503, 637956, 772665, 961858, 996374, 1070950, 1092245, 1101005, 1144589, 1315936, 1487448, 1781884, 1793515, 1832314, 1853695, 1928272]
[204205, 301220, 450112, 1203689]
[223532, 502419, 738534, 1261973, 1346969]
[229711, 283040, 344336, 788664, 828107, 951482, 1312029, 1874728, 1965358]
